In [18]:
import pandas as pd
import numpy as np
import plotly.express as px
from pathlib import Path

data1 = Path("../data/processed/analytics_table.parquet")
figuers1 = Path("reports/figures")
figuers1.mkdir(parents=True, exist_ok=True)




In [14]:
df = pd.read_parquet("../data/processed/analytics_table.parquet")
df.head(7)

,order_id,user_id,amount,quantity,created_at,status,status_clean,amount__isna,quantity__isna,date,year,month,dow,hour,country,signup_date,amount_winsor
0,A0001,1,12.5,1.0,2025-12-01 10:05:00+00:00,Paid,paid,False,False,2025-12-01,2025.0,2025-12,Monday,10.0,SA,2025-11-15,12.500
1,A0002,2,8.0,2.0,2025-12-01 11:10:00+00:00,paid,paid,False,False,2025-12-01,2025.0,2025-12,Monday,11.0,SA,2025-11-20,8.135
2,A0003,3,NaN,1.0,2025-12-02 09:00:00+00:00,Refund,refund,True,False,2025-12-02,2025.0,2025-12,Tuesday,9.0,AE,2025-11-22,NaN
3,A0004,1,25.0,NaN,2025-12-03 14:30:00+00:00,PAID,paid,False,True,2025-12-03,2025.0,2025-12,Wednesday,14.0,SA,2025-11-15,25.000
4,A0005,4,100.0,1.0,NaT,paid,paid,False,False,None,NaN,<NA>,None,NaN,SA,2025-11-25,97.750


In [25]:
df= pd.read_parquet(data1)
print("rows number : ",len(df))
print("cols number :", len(df.columns))
print(df.dtypes.head(17))

print(df.isna().sum().sort_values(ascending=False).head(10))

rows number :  5
cols number : 17
order_id               string[python]
user_id                         Int64
amount                        float64
quantity                      float64
created_at        datetime64[ns, UTC]
status                 string[python]
status_clean                   object
amount__isna                     bool
quantity__isna                   bool
date                           object
year                          float64
month                  string[python]
dow                            object
hour                          float64
country                        object
signup_date                    object
amount_winsor                 float64
dtype: object
amount           1
amount_winsor    1
quantity         1
created_at       1
hour             1
dow              1
month            1
year             1
date             1
order_id         0
dtype: int64


# Questions , Results

1. How does total revenue change over time?
2. How does revenue differ by country?
3. What is the distribution of order amounts (winsorized)?
4. Is the refund rate different across countries?